In [242]:
from IPython.display import display, Image, clear_output, HTML
import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import json
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [243]:
#this is the function that sends the data to the google form

def send_to_google_form(data_dict, form_url):

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [244]:
#registers the clicking of a button

def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [245]:
#this function will be triggered when someone clicks the button in the get_details section  

def submit_button():
    confirm_btn = widgets.Button(description = "Confirm")
    display(confirm_btn)
    confirm_btn.on_click(register_btn_event)
    wait_for_event()
    clear_output(wait=False)
    return

In [246]:
#used to display a message for the user with a countdown feature in it

def timer(message):
    for i in range(3, 0, -1):
        clear_output(wait=True)
        print(message.format(i))
        time.sleep(1)
    clear_output(wait=True)

In [247]:
#the intro for each of the three questions in the test

def intro(url, message):
    clear_output(wait=False)

    print(message)
    time.sleep(1)

    timer('Image showing in {} seconds')
    
    grid1 = Image(f'{url}', width=300)
    display(grid1)
    time.sleep(20)
    clear_output(wait=False)
    time.sleep(1)
    
    timer('Test starting in {} seconds')

In [248]:
#waits for the clicking of a button

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)

In [249]:
#separate function for the clicking of a button within the question, this will also update the number of correct and incorrect answers

def register_event(btn, correct_ans):

    global correct, incorrect
    
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()

    if btn.description == correct_ans:
        correct += 1
        answers.append(1)
    else:
        incorrect += 1
        answers.append(0)
    return

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

In [260]:
#gets the details of the users

def get_details():

    global person_name, gender, age

    print('In this test you will be given 3 different grids, each grid will be shown for 20 seconds and will have 5 questions associated with it.' + 
    '\nBoth speed and accuracy will be taken into account when calculating your mark')
    time.sleep(15)
    clear_output(wait=False)
    
    display(HTML("<span>Input your name here</span>"))
    person_name = input()
    clear_output(wait=False)
    
    print('What is your age?')
    age_dropdown = widgets.Dropdown(options=[('Select Age', None)] + [(str(age), age) for age in range(18, 30)], value=None)
    
    display(age_dropdown)
    submit_button()

    gender_radiobuttons = widgets.RadioButtons(options=['Male', 'Female', 'Other'], description='What is your gender?', disabled=False)
    
    display(gender_radiobuttons)
    submit_button()

    age = age_dropdown.value
    gender = gender_radiobuttons.value


In [251]:
#represents a question for the test

def question(question, btn1, btn2, btn3, btn4, correct_ans):

    global total_time, correct, incorrect 

    key = Image('key.png', width=1000)
    display(key)

    display(HTML(f'<span>{question}</span>'))
    start_time = time.time()

    btn1 = widgets.Button(description=btn1)
    btn2 = widgets.Button(description=btn2)
    btn3 = widgets.Button(description=btn3)
    btn4 = widgets.Button(description=btn4)

    btn1.on_click(lambda btn: register_event(btn, correct_ans)) 
    btn2.on_click(lambda btn: register_event(btn, correct_ans)) 
    btn3.on_click(lambda btn: register_event(btn, correct_ans))
    btn4.on_click(lambda btn: register_event(btn, correct_ans))

    panel = widgets.HBox([btn1, btn2, btn3, btn4])
    display(panel)

    wait_for_event()
    clear_output(wait=False)
    
    end_time = time.time()
    time_taken = end_time - start_time
    times.append(time_taken)
    total_time += time_taken
      

In [262]:
#calculates the score for the user

def calculate_score():

    global percent, score
    
    clear_output(wait=False)
    
    percent = correct / (incorrect + correct) * 100
    score = percent 
    
    time_over = total_time - 90
    penalty_periods = time_over // 10
    
    for i in range(int(penalty_periods)):
        score -= 5  
    
    print(f'You got {round(percent, 1)}% in {round(total_time, 1)} seconds.')
    
    if score > 0:
        print(f'This gives you an overall score of {round(score, 1)}')
    else:
        print('You failed the test')

In [264]:
#whole test

def run_test():
    
    global correct, incorrect, times, answers, total_time, score, percent

    get_details()
    
    intro('grid1.png', 'Level easy')
    
    question('What was between the rectangle and triangle?', 'square', 'circle', 'cross', 'star', 'circle')
    question('What colour was the square?', 'red', 'green', 'orange', 'blue', 'red')
    question('Where was the star?', 'bottom right', 'bottom middle', 'top right', 'bottom left', 'bottom left')
    question('What was purple?', 'circle', 'cross', 'rectangle', 'square', 'rectangle')
    question('What was above the cross?', 'triangle', 'circle', 'rectangle', 'square', 'triangle')
    
    intro('grid2.png', 'Level medium')
    
    question('What was the colour of the shape in the top right?', 'pink', 'purple', 'yellow', 'red', 'yellow')
    question('What was the middle left shape?', 'triangle', 'pentagon', 'star', 'rectangle', 'triangle')
    question('Where was the pentagon?', 'top right', 'top left', 'bottom left', 'bottom right', 'top left')
    question('What was between the circle and the diamond?', 'cross', 'ellipse', 'star', 'square', 'square')
    question('How many of the shapes had rounded edges?', 'one', 'two', 'three', 'four', 'two')
    
    intro('grid3.png', 'Level hard')
    
    question('Where was the green circle?', 'bottom middle', 'bottom left', 'bottom right', 'middle right', 'bottom right')
    question('In the top right the big shape was a triangle, what was the small shape?', 'triangle', 'square', 'rectangle', 'cross', 'triangle')
    question('What colour was the central big shape?', 'blue', 'red', 'orange', 'yellow', 'yellow')
    question('What was the small shape in the middle of the big triangle and big cross?', 'pentagon', 'square', 'rectangle', 'star', 'square')
    question('Are there any cells with the two shapes being yellow and blue?','x', 'yes', 'no', 'x', 'yes')

    calculate_score()
    
    upload_data()


In [253]:
#sending data to the google form

def upload_data():
    
    results_data = {
        'filename': ['grid1.png', 'grid1.png', 'grid1.png', 'grid1.png', 'grid1.png', 
                    'grid2.png', 'grid2.png', 'grid2.png', 'grid2.png', 'grid2.png', 
                    'grid3.png', 'grid3.png', 'grid3.png', 'grid3.png', 'grid3.png'],
        'time': times,
        'answer': answers
    }
    
    myresults = pd.DataFrame(results_data)
    results_json = myresults.to_json()
    
    
    data_dict = {
        'name': person_name,
        'gender': gender,
        'age': age,
        'percent': percent,
        'total time': total_time,
        'score': score,
        'results': results_json 
    }
    
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSf6VxaewHHjuZ0vbNP-7zDOg1I_zJ8F9dhOmJF7KWztlAuJ5A/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url)

In [265]:
#click here to do the test

correct = 0
incorrect = 0
times = []
answers = [] 
total_time = 0
score = 0
percent = 0

run_test()

You got 26.7% in 5.9 seconds.
This gives you an overall score of 26.7
